?  =  variable name, possibly with indexing/slicing  
??? = function / class name  
?????? = complex expression containing variables, functions etc  
???B??? = complex expression that must contain "B"  

# Extracting behavior from mouse face videos

### Load in mouse face videos

Using the `pyav` library, load in the first 3000 frames of this movie. It is a gray-scale movie so we only take one channel of the RGB output.

In [ ]:
import av, os
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np
import time
from scipy.stats import skew
from scipy.sparse.linalg import eigsh
from facemap import utils

### PUT HERE THE PATH TO THE MOVIE
root = '/home/neuraldata/data/meso/'
mouse_name = 'TX39'
moviename = os.path.join(root, mouse_name, 'cam1_TX39.avi')

# open the video for reading
container = av.open(moviename)
container.streams.video[0].thread_type = 'AUTO'
nframes = container.streams[0].duration

# read the movie frame by frame
k=0
for frame in container.decode(video=0):
    array = frame.to_ndarray(format='rgb24')
    array = array[:,:,0] # take the first channel of the frame (red)
    if k==0:
        # initialize imgs to an array of zeros of type uint8, size 3000 by movie height by movie width
        imgs = np.???((?, ?, ?), '???') 
    imgs[k] = array
    k+=1
    if k==3000:
        break   
    
# convert imgs to float32 (or "'"single")
imgs = np.???(imgs)

### Compute the motion energy

To capture behavior, we do not necessarily care where the mouse's whisker is - rather we care if it's moving and in what patterns. To compute movement features we start by: 1) taking the difference between frames, 2) taking the absolute value of this because we want overall movement, regardless of sign. This absolute value of the difference is the **motion energy**.

In [ ]:
# motion energy
motion = np.???(np.???(imgs, axis=0))

In [ ]:
# show some frames

fig = plt.figure(figsize=(15,5))

# example frame
ax=fig.add_subplot(1,3,1)
ax.imshow(?, cmap='gray') # display frame 100
ax.set_title('example frame')

# average mouse face
ax=fig.add_subplot(1,3,2)
ax.imshow(np.???(imgs, axis=0), cmap='gray') # display the mean frame
ax.set_title('average frame')

# average motion energy
ax=fig.add_subplot(1,3,3)
ax.imshow(np.mean(?, axis=0), cmap='gray') # display the mean motion energy
ax.set_title('average motion energy')

plt.show()

### Reduce dimensionality of motion energy

Motion energy has dimensions = number of frames by number of pixels. We want to only keep the top 100 principal components of this motion energy. Let's compute the top 100 "spatial" components of the data.

In [ ]:
motion = np.reshape(motion, (motion.shape[0],-1))
motion = motion - ?????? # subtract the mean across time

# take the principal components of these mean-centered motion energy frames
from sklearn.decomposition import PCA
ncomps = 100
# fit a PCA model to the motion variable
pca_model = ???ncomps???

print('spatial components of size (%d, %d)'%pca_model.components_.shape)

We can plot these spatial components. Observe that most of the variation occurs in the whisker pad.

In [ ]:
NT, Ly,Lx = ?.shape # movie dimensions
motMask = np.reshape(pca_model.components_, (?, Ly, Lx)) # reshape PCA components
plt.figure(figsize=(15,8))
for i in range(15):
    ax=plt.subplot(3,5,i+1)
    # plot the i-th motion energy mask, use the blue-white-red colormap, set saturation limits of -0.01 and 0.01
    ax.imshow(?, ?=?, ?=-?, ?=?)
    ax.axis('off')
plt.show()

### Project spatial components onto all frames

To compute the low-dimensional representation of the motion energy across time, we need to now PROJECT these spatial components onto the the motion energy for each frame.

The principal components are the same as the $U$ in singular value decomposition, where the singular value decomposition is
$$ X \approx USV^\top $$

Let's multiply $U^\top$ on both sides, recall that $U U^\top = I$ because $U$ is an orthonormal matrix.
$$ U^\top X = S V^\top $$

So $U^\top X$ gives the right singular vectors scaled by the variance of each component. We will use this instead of using $S^{-1} U^\top X$. Why might we want them to be weighted by their true variance in the video?

In [ ]:
# compute the temporal coefficients for the principal components
motSVD = model_pca.??? @ ?.T # project the raw movie into the spatial components

In [ ]:
# PLOT TRACES

plt.figure(figsize=(15,3))

colormap = plt.cm.magma
colors   = colormap(np.linspace(0,1,10)) # create an array of colors from one end to the other of the colormap
for n in range(10):
    plt.plot(motSVD[n] + ?*?, color=cmap[n], zorder=10-n) # spread out the traces in Y for visualization
plt.show()